In [14]:
#Packages

import json
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import requests
from datetime import datetime, timedelta


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Go get geckodriver from : https://github.com/mozilla/geckodriver/releases

In [2]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder)
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")
    
   # profile.add_extension('/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/booking/ublock_origin-1.55.0.xpi')


    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True)

    options = Options()
    options.profile = profile
    return options


def start_up(link, dfolder, geko_path,donwload=True):
    # geko_path='/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/geckodriver'
    # download_path='./downloads'
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder,donwload)
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    # Enter the website address here
    browser.get(link)
    time.sleep(5)  # Adjust sleep time as needed
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

## Opening Booking.com

In [3]:
# lets open booking:

dfolder='./downloads'
geko_path='/Users/mikelgallo/repos2/text_mining/Scraping_UN/books/geckodriver'
link='https://www.booking.com/index.es.html'

browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)

#### Pending - Removing popups

#### Opening Search_box

In [4]:
browser.find_element(by='xpath',value= '//input[@id=":re:"]').click()

#### Inputing destination

In [5]:
place = input('Where do you want to go?')
search1 = browser.find_element(by='xpath',value='//*[@id=":re:"]')
search1.send_keys(place)

Where do you want to go? Valencia


#### Opening Dates table

In [18]:
#Use CSS Selector Copy
css='button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'
browser.find_element('css selector',css).click()

#### Selecting Dates

In [23]:
#Selecting Dates table and retrieving date information
def extract_dates():
    dates_f = []
    path='//div[@id="calendar-searchboxdatepicker"]//table[@class="eb03f3f27f"]//tbody//td[@class="b80d5adb18"]//span[@class="cf06f772fa"]'
    dates = browser.find_elements('xpath',path)
    for date in dates:
        dates_f.append(date)
    return dates_f

In [24]:
dates_f = extract_dates()

for date in dates_f:
    print(date.get_attribute("data-date"))

2024-01-29
2024-01-30
2024-01-31
2024-02-01
2024-02-02
2024-02-03
2024-02-04
2024-02-05
2024-02-06
2024-02-07
2024-02-08
2024-02-09
2024-02-10
2024-02-11
2024-02-12
2024-02-13
2024-02-14
2024-02-15
2024-02-16
2024-02-17
2024-02-18
2024-02-19
2024-02-20
2024-02-21
2024-02-22
2024-02-23
2024-02-24
2024-02-25
2024-02-26
2024-02-27
2024-02-28
2024-02-29


In [25]:
for date in dates_f:
    if date.get_attribute("data-date") == "2024-05-29":
        time.sleep(1)
        date.click()
    if date.get_attribute("data-date") == "2024-06-02":
        time.sleep(1)
        date.click()
        break

In [11]:
#Returning pages
#for i in range(1,6):
    #if i == 1:
        #x_path = '/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/button[1]'
    #else:
        #x_path = '/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/button[1]'
    #browser.find_element(by='xpath',value=x_path).click()
    #time.sleep(2)

#### Inputing Dates 

### Searching

In [10]:
my_xpath='/html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span'

check_obscures(browser,my_xpath , type='xpath')
check_and_click(browser,my_xpath , type='xpath')

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/E

#### Retrieve data

### Getting the number of Pages

In [11]:
def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath','//div[@data-testid="pagination"]//li[@class="b16a89683f"]')
    return(int(a[-1].text))


pages = get_number_pages(browser)

print(pages)

40


#### Creating Click button

In [12]:
#Click next page
x_path='/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[4]/div[2]/nav/nav/div/div[3]/button/span/span'
#browser.find_element(by='xpath',value=x_path).click()

#### Dates to look for

In [17]:
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import re

num_pages = 0
df = []
#Click page path
x_path = '/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[4]/div[2]/nav/nav/div/div[3]/button/span/span'

for i in range(1, 5):
    time.sleep(7)
    links = []
    try:
        link = browser.find_elements('xpath', "//div[@class='d6767e681c']//a")
        for i in link:
            links.append(i.get_attribute('href'))
    except NoSuchElementException:
        links.append(np.nan)

    hotels_list = []
    try:
        hotels = browser.find_elements('xpath', '//div[@class="f6431b446c a15b38c233"]')
        for hotel in hotels:
            hotels_list.append(hotel.text)
    except NoSuchElementException:
        hotels_list.append(np.nan)

    prices_list = []
    try:
        prices = browser.find_elements('xpath', '//span[@class="f6431b446c fbfd7c1165 e84eb96b1f"]')
        for price in prices:
            prices_list.append(price.text)
    except NoSuchElementException:
        prices_list.append(np.nan)

    districts_list = []
    try:
        districts = browser.find_elements('xpath', '//span[@class="aee5343fdb def9bc142a"]')
        for i in districts:
            if ', valencia' in i.text.lower():
                districts_list.append(i.text)
    except NoSuchElementException:
        districts_list.append(np.nan)
    except:
        districts_list.append(np.nan)

    # Retrieving only center distances
    distance_center = []
    try:
        distance = browser.find_elements('xpath', '//div[@class="abf093bdfe ecc6a9ed89"]//span[@class="f419a93f12"]')
        for i in distance:
            if 'centro' in i.text:
                distance_center.append(i.text)
    except NoSuchElementException:
        distance_center.append(np.nan)
    except:
        distance_center.append(np.nan)
## Ratings and Comments
    ratings_list = []
    try:
        ratings = browser.find_elements('xpath', '//div[@class="a3b8729ab1 d86cee9b25"]')
        for rate in ratings:
            ratings_list.append(rate.text)
    except NoSuchElementException:
        ratings_list.append(np.nan)
    comment_list = []
    try:
        comentarios = browser.find_elements('xpath', '//div[@class="abf093bdfe f45d8e4c32 d935416c47"]')
        for com in comentarios:
            comment_list.append(com.text)
    except:
        comment_list.append(np.nan)

    for a, b, c, d, e, f, g in zip(hotels_list, distance_center, districts_list, prices_list,ratings_list,comment_list, links):
        row_data = {'Hotels': a, 'Distance': b, 'District': c, 'Price': d, 'Rating': e, 'Comments': f, 'Link': g}
        print(row_data)
        df.append(row_data)

    wait = WebDriverWait(browser, 10)  # Adjust the timeout as needed
    next_button = wait.until(EC.element_to_be_clickable((By.XPATH, x_path)))
    next_button.click()
    num_pages += 1


In [16]:
df_1 = pd.DataFrame(df)
df_1.shape

(0, 0)

In [98]:
df_new[100:200]

,Hotels,Distance,District,Price,Rating,Comments,Link
100,Hotel El Palace Barcelona,a 500 m del centro,"Sants-Montjuïc, Barcelona",€ 5.212,"9,3",1.921 comentarios,https://www.booking.com/hotel/es/ritzbcn.es.ht...
101,Rent Top Apartments Forum,"a 5,3 km del centro","Gracia, Barcelona",€ 3.163,"8,9",7 comentarios,https://www.booking.com/hotel/es/rent-top-apar...
102,Cosmo Apartments Sants,a 3 km del centro,"Eixample, Barcelona",€ 1.634,"7,8",1.753 comentarios,https://www.booking.com/hotel/es/just-style.es...
103,Hotel Casa Fuster G.L Monumento,"a 1,6 km del centro","Eixample, Barcelona",€ 4.892,"8,9",1.464 comentarios,https://www.booking.com/hotel/es/hotelescenter...
104,Hotel Royal Passeig de Gracia,a 1 km del centro,"Eixample, Barcelona",€ 2.310,"8,5",3.706 comentarios,https://www.booking.com/hotel/es/royal-passeig...
...,...,...,...,...,...,...,...
195,The Hipstel Parallel,"a 1,6 km del centro","Gracia, Barcelona",€ 362,"9,0",568 comentarios,https://www.booking.com/hotel/es/futura-casa-l...
196,Mellow Barcelona,"a 4,3 km del centro","Gracia, Barcelona",€ 310,"7,9",381 comentarios,https://www.booking.com/hotel/es/mellow-eco-ho...
197,Factory Hostels Barcelona,a 3 km del centro,"Eixample, Barcelona",€ 419,"6,8",947 comentarios,https://www.booking.com/hotel/es/la-flor-de-ga...
198,Rocket Hostels Gracia,"a 3,1 km del centro","Les Corts, Barcelona",€ 463,"8,0",2.872 comentarios,https://www.booking.com/hotel/es/the-hostel-bo...


In [119]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

num_pages = 0
df = []
x_path = '/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[4]/div[2]/nav/nav/div/div[3]/button/span/span'
for i in range(1, pages+1):
    time.sleep(5)
    hotels_list = []
    try:
        hotels = browser.find_elements('xpath', '//div[@class="f6431b446c a15b38c233"]')
        for hotel in hotels:
            hotels_list.append(hotel.text)
    except NoSuchElementException:
        hotels_list.append(np.nan)
    stars = []
    try:
        elements = browser.find_elements('xpath', '//div[@class="b3f3c831be"][@aria-label]')
        stars = [element.get_attribute('aria-label') for element in elements]
    except NoSuchElementException:
        stars.append(np.nan)
    except:
        stars.append(np.nan)
    for a, b in zip(hotels_list, stars):
        row_data = {'Hotels': a, 'Stars': b}
        print(row_data)
        df.append(row_data)
    wait = WebDriverWait(browser, 10)  # Adjust the timeout as needed
    next_button = wait.until(EC.element_to_be_clickable((By.XPATH, x_path)))
    next_button.click()
    num_pages += 1

{'Hotels': 'H La Paloma Adults Only', 'Stars': '2 de 5'}
{'Hotels': 'Le Meridien Barcelona', 'Stars': '5 de 5'}
{'Hotels': 'Hotel Barcelona Center', 'Stars': '4 de 5'}
{'Hotels': 'Vincci Maritimo', 'Stars': '4 de 5'}
{'Hotels': 'Barcelona City Apartment', 'Stars': '3 de 5'}
{'Hotels': 'Vincci Bit', 'Stars': '4 de 5'}
{'Hotels': 'Leevin Student Barcelona', 'Stars': '1 de 5'}
{'Hotels': 'Travelodge Barcelona Poblenou', 'Stars': '3 de 5'}
{'Hotels': 'Barcelona Central Apartments', 'Stars': '5 de 5'}
{'Hotels': 'Gran Hotel La Florida G.L Monumento', 'Stars': '2 de 5'}
{'Hotels': 'Barcelona City North Hostal', 'Stars': '4 de 5'}
{'Hotels': 'Suites Avenue', 'Stars': '5 de 5'}
{'Hotels': 'Majestic Hotel & Spa Barcelona GL', 'Stars': '3 de 5'}
{'Hotels': 'Sunotel Aston', 'Stars': '2 de 5'}
{'Hotels': 'Hostal Bcn 46', 'Stars': '5 de 5'}
{'Hotels': 'Torre Melina, a Gran Meliá Hotel', 'Stars': '2 de 5'}
{'Hotels': 'Hostal Boqueria', 'Stars': '3 de 5'}
{'Hotels': 'Barcelona Apartment Republica', '

In [120]:
df_2 = pd.DataFrame(df)
df_2.shape

(172, 2)

In [ ]:
# Merging dataframes


In [178]:
descriptions = []
path = '/Users/mikelgallo/repos2/Bookings/mikel_folder'
for i in df_new['Link']:
    URL = f'{i}'

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    div = soup.find('div', {'id': 'property_description_content'})

    # Find all p elements with a specific class within the div
    specific_class_p_elements = div.find_all('p', class_='a53cbfa6de b3efd73f69')
    
    for p in specific_class_p_elements:
        descriptions.append(p.get_text(strip=True))
    #descriptions.append(specific_class_p_elements)
    
    # Find all p elements within the div
    #p_elements = div.find_all('p')
    
    # Extract text from each p element and append to descriptions
    # Define a custom filter to include only <p> tags and exclude <a> tags
    # Find all p elements within the div excluding a tags

In [192]:
links = [my_dict.get('Link') for my_dict in df]
len(links)

25

In [193]:
descriptions = []
path = '/Users/mikelgallo/repos2/Bookings/mikel_folder'
links = [my_dict.get('Link') for my_dict in df]
for i in links:
    URL = f'{i}'

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    div = soup.find('div', {'id': 'property_description_content'})

    # Find all p elements with a specific class within the div
    specific_class_p_elements = div.find_all('p', class_='a53cbfa6de b3efd73f69')
    
    for p in specific_class_p_elements:
        descriptions.append(p.get_text(strip=True))

In [194]:
df_new['descriptions'] = descriptions

In [195]:
df_new

,Hotels,Ratings,Distance,District,Price,Link,descriptions
0,H10 Casa Mimosa 4* Sup,"9,2","a 1,2 km del centro","Eixample, Barcelona",€ 1.156,https://www.booking.com/hotel/es/h10-casa-mimo...,El H10 Casa Mimosa está situado en un edificio...
1,Sonder Casa Luz,"8,4",a 450 m del centro,"Eixample, Barcelona",€ 1.787,https://www.booking.com/hotel/es/casa-luz-barc...,El Sonder Casa Luz se encuentra en Barcelona y...
2,MERCE APARTMENT FORUM BARCELONA,"7,6","a 5,5 km del centro","Ciutat Vella, Barcelona",€ 662,https://www.booking.com/hotel/es/merce-apartme...,"MERCE APARTMENT FORUM BARCELONA, que cuenta co..."
3,H La Paloma Adults Only,"8,9","a 0,7 km del centro","Gracia, Barcelona",€ 712,https://www.booking.com/hotel/es/h-la-paloma.e...,The H La Paloma is located in the Raval neighb...
4,Boutique Apartments 23 Barcelona,"8,9","a 1,6 km del centro","Sants-Montjuïc, Barcelona",€ 667,https://www.booking.com/hotel/es/23-barcelona....,Boutique Apartments 23 Barcelona está a menos ...
5,Onefam Sants,"7,3","a 3,6 km del centro","Sant Martí, Barcelona",€ 404,https://www.booking.com/hotel/es/hostel-one-sa...,El Onefam Sants es un animado albergue social ...
6,Travelodge Barcelona Poblenou,"8,2","a 2,9 km del centro","Sant Martí, Barcelona",€ 334,https://www.booking.com/hotel/es/travelodge-ba...,Este moderno hotel cuenta con 250 amplias y co...
7,Catalonia Sagrada Familia,"8,3","a 2,6 km del centro","Sant Martí, Barcelona",€ 441,https://www.booking.com/hotel/es/cataloniaarag...,El Catalonia Sagrada Familia se halla a 15 min...
8,Vincci Maritimo,"8,4","a 4,2 km del centro","Sant Martí, Barcelona",€ 522,https://www.booking.com/hotel/es/vinccimaritim...,El Vincci Marítimo se encuentra a 10 minutos a...
9,Vincci Bit,"8,7","a 4,4 km del centro","Sant Martí, Barcelona",€ 483,https://www.booking.com/hotel/es/vincci-bit.es...,Vincci Bit se encuentra a tan solo 10 minutos ...


In [35]:
browser.quit()